In [1]:
import pandas as pd
from collections import Counter
import spacy
import pytextrank
import json

# Create comment wise JSON

In [243]:
# df = pd.read_json('communitycrit_revised.json')
# df = pd.read_json('env_canada_final.json')

In [244]:
data = []
cid = 0;

for i in df["ideas"]:
    idea_name = i["name"]
    idea_id = i["serial"]
    for t in i["tasks"]:
        for c in t["comments"]:
            data_packet = {
                "comment_id": cid,
                "comment": c["comment"],
                "related_to": (c["question"] if c["task_id"] == 12 else t["name"]),
                "discussion_id": idea_id,
                "discussion_name": idea_name,
                "user_id": c["user_id"],
                "emotion": "",
                "discourse": "",
                "dominant_topic": "",
                "topic_phrase": "",
                "topic_keywords": "",
                "lemmas": ""
            }
            cid += 1
            data.append(data_packet)

In [245]:
emo_df = pd.read_json('cc_preds_emo_bert.json')
# emo_df = pd.read_json('envcanada_preds_emo_bert.json')

emo_df.head(10)

for idx, row in emo_df.iterrows():
    data[idx]["emotion"] = row["emotion"]

In [246]:
disc_df = pd.read_json('cc_preds_bert.json')
# disc_df = pd.read_json('envcanada_preds_bert.json')

disc_df.head(10)

for idx, row in disc_df.iterrows():
    data[idx]["discourse"] = row["discourse"]

In [247]:
topic_df = pd.read_json('cc_sent.json')
# topic_df = pd.read_json('env_sent.json')

for idx, row in topic_df.iterrows():
    data[idx]["dominant_topic"] = row["dominant_topic"]
    data[idx]["topic_keywords"] = row["keywords"].split(", ")

In [248]:
lemma_df = pd.read_json('cc_lemmas.json')
# lemma_df = pd.read_json('env_lemmas.json')

for idx, row in lemma_df.iterrows():
    data[idx]["lemmas"] = row["lemmas"]

In [249]:
def get_keyphrase(prop_rows, t):
    text = ""
    rows = prop_rows.loc[prop_rows['dominant_topic'] == t]
    for c in rows["comment"]:
        text = text + " " + c
        
    nlp = spacy.load("en_core_web_sm")
    tr = pytextrank.TextRank(logger=None)
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
    doc = nlp(text)
    
#     print(doc._.phrases, "\n\n")
    
    if (doc._.phrases):
        return doc._.phrases[0].text
    else:
        return rows.iloc[0]["keywords"].split(", ")[0]
    

topic_phrases = []
proposals = Counter(topic_df["idea"])
for p in proposals:
    proposal_wise_topic_phrases = {
        "proposal": p,
    }
    prop_rows = topic_df.loc[topic_df['idea'] == p]
    topics = Counter(prop_rows["dominant_topic"])
    print(topics)
    for t in topics:
        proposal_wise_topic_phrases["topic"] = t
        proposal_wise_topic_phrases["topic_phrase"] = get_keyphrase(prop_rows, t)
        topic_phrases.append(proposal_wise_topic_phrases.copy())

Counter({1: 8, 0: 7, 4: 7, 3: 7, 5: 4, 2: 3})
Counter({1: 12, 0: 11})
Counter({4: 9, 2: 6, 1: 4, 0: 3, 3: 2, 5: 2})
Counter({1: 8, 0: 7, 3: 6, 2: 5})
Counter({0: 8, 2: 6, 4: 5, 1: 4, 3: 3, 5: 2})
Counter({1: 6, 0: 5, 2: 3, 3: 2})
Counter({0: 6, 4: 5, 2: 4, 3: 2, 1: 2, 5: 1})
Counter({6: 6, 2: 6, 4: 4, 1: 3, 0: 2, 3: 2, 5: 2})
Counter({0: 10, 4: 6, 1: 6, 3: 5, 5: 4, 2: 3})
Counter({1: 1})
Counter({0: 11, 1: 10})
Counter({0: 9, 2: 6, 3: 5, 1: 1})
Counter({0: 3, 1: 2, 3: 2})
Counter({0: 6, 3: 5, 1: 5, 2: 3})
Counter({1: 3, 0: 3})
Counter({0: 2, 1: 1})
Counter({1: 2, 0: 2})
Counter({0: 3, 1: 2})
Counter({4: 5, 2: 5, 5: 4, 3: 3, 1: 2, 0: 2, 6: 1, 7: 1})


In [250]:
for idx, row in topic_df.iterrows():
    for tp in topic_phrases:
        if(row["idea"] == tp["proposal"] and row["dominant_topic"] == tp["topic"]):
            data[idx]["topic_phrase"] = tp["topic_phrase"]

In [251]:
# with open('data_env.json', 'w') as outfile:
#     json.dump(data, outfile)
    
with open('data_cc.json', 'w') as outfile:
    json.dump(data, outfile)

In [252]:
# x = topic_df.loc[topic_df['idea'] == 'Public Art']
# y = x.loc[x['dominant_topic'] == 1]

# len(y)

In [253]:
# for i, row in y.iterrows():
#     print(row.comment)

# Create proposal wise JSON

In [2]:
# df = pd.read_json('data_cc.json')
df = pd.read_json('data_env.json')

In [3]:
df

,comment,comment_id,discourse,discussion_id,discussion_name,dominant_topic,emotion,lemmas,related_to,topic_keywords,topic_phrase,user_id
0,"1)quite often, will not purchase something tha...",0,disagreement,1,Sustainable Design and Production,1,concern,"[quite, often, purchase, pack, especially, reg...",Share your comment,"[plastic, waste, environment, problem, issue, ...",their waste plastic,Carmen Stanek
1,"Yes we need to reduce plastics, but why are t...",1,question,1,Sustainable Design and Production,1,anger,"[need, reduce, plastic, barrier, recycle, need...",Share your comment,"[plastic, waste, environment, problem, issue, ...",their waste plastic,Stephen Green
2,"Everyone talks about plastics being ""recyclabl...",2,disagreement,1,Sustainable Design and Production,6,anger,"[talk, plastic, recyclable, yet, many, packagi...",Share your comment,"[government, consumer, reduce, ban, change, in...",their plastic products,David McBeth
3,1. The amount and need for packaging. \n2. Pla...,3,disagreement,1,Sustainable Design and Production,4,anger,"[amount, need, packaging, place, package, good...",Share your comment,"[packaging, product, package, item, option, go...",their plastic packaging,Peter Battistoni
4,These are little ideas: I am at a loss when I ...,4,disagreement,1,Sustainable Design and Production,0,anger,"[little, idea, loss, go, supermarket, way, buy...",Share your comment,"[container, make, bottle, bring, glass, water,...",their single use plastics,Meggie Ross
5,I always consider packaging when I am purchasi...,5,question,1,Sustainable Design and Production,4,neutral,"[always, consider, package, purchase, thing, t...",Share your comment,"[packaging, product, package, item, option, go...",their plastic packaging,ruth quirk
6,1) I try to buy bulk whenever possible.\n2) I ...,6,answer,1,Sustainable Design and Production,2,neutral,"[try, buy, bulk, whenever_possible, try, buy, ...",Share your comment,"[bag, food, buy, reusable, store, reuse, groce...",plastic bags,Jami Koehl
7,We have to start legislating industry as if th...,7,elaboration,1,Sustainable Design and Production,0,anger,"[start, legislate, industry, produce, people, ...",Share your comment,"[container, make, bottle, bring, glass, water,...",their single use plastics,Allen Leier
8,Media has had a great effect on consumer behav...,8,answer,1,Sustainable Design and Production,6,anger,"[medium, great, effect, consumer, behavior, lo...",Share your comment,"[government, consumer, reduce, ban, change, in...",their plastic products,Judy Smith
9,Make the cost of items in plastic packaging mu...,9,elaboration,1,Sustainable Design and Production,4,anger,"[make, cost, item, package, much, high, item, ...",Share your comment,"[packaging, product, package, item, option, go...",their plastic packaging,Judy Smith


In [4]:
def get_good_keywords(topic_rows, keywords):
    good_kws = []
    for i in keywords:
        cnt = 0
        for idx, row in topic_rows.iterrows():
            if (i in row["comment"] or i in row["lemmas"]):
                cnt += 1
        if(cnt > 0):
            good_kws.append({"keyword": i, "count": cnt})
    
    x = sorted(good_kws, key=lambda x: x["count"], reverse=True)
    return ([d['keyword'] for d in x])
    
def get_topic_discourse(topic_rows):
    discourse_sets = []
    discourses = ["question", "answer", "announcement", "elaboration", "appreciation", "agreement", "disagreement", "negativereaction", "humor", "other"]
    for j in discourses:
        discourse_set_elements = {
            "discourse_name" : j,
            "count" : Counter(topic_rows["discourse"])[j] if Counter(topic_rows["discourse"])[j] > 0 else 0,
            "color" : get_blue(len(topic_rows), Counter(topic_rows["discourse"])[j])
        }
        discourse_sets.append(discourse_set_elements)
    
    return(discourse_sets)
    
    
def get_blue(total, count):
    blueSpectrum = ["#f6feff","#f7fbff","#deebf7","#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
    color = round((((count / total)) * 2) * 9) if (round((((count / total)) * 2) * 9)) < 10 else 9
    return blueSpectrum[color]
        
prop_data = []
    
for i in Counter(df["discussion_name"]).keys(): 
    print(i)
    rows = df.loc[df["discussion_name"] == i]
    topic_coverage = round(len(rows) * .15)
#     topic_coverage = round(len(rows) * .10)
    prop_data_packet = {
        "discussion_id" :  rows["discussion_id"].tolist()[0],
        "discussion_name": rows["discussion_name"].tolist()[0],
        "number_of_comments": len(rows),
        "topics": [],
        "discourses": [],
        "comments" : []
    }
    
    topic_sets = []
    for j in Counter(rows["topic_phrase"]).keys():
        topic_rows = rows.loc[rows["topic_phrase"] == j]
        if(len(topic_rows) >= topic_coverage):
            topic_set_elements = {
                "topic_phrase" : j,
                "topic_keywords" : get_good_keywords(topic_rows, topic_rows["topic_keywords"].tolist()[0]),
                "topic_sent_count" : len(topic_rows),
                "topic_sent_perc" : round((len(topic_rows)/len(rows)) * 100),
                "topic_discourses" : get_topic_discourse(topic_rows)
            }
            topic_sets.append(topic_set_elements)
    
    prop_data_packet["topics"] = sorted(topic_sets, key=lambda x: x["topic_sent_count"], reverse=True)
    
#     print(prop_data_packet["topics"])
    
    discourse_sets = []
    discourses = ["question", "answer", "announcement", "elaboration", "appreciation", "agreement", "disagreement", "negativereaction", "humor", "other"]
    for j in discourses:
        discourse_set_elements = {
            "discourse_name" : j,
            "count" : Counter(rows["discourse"])[j] if Counter(rows["discourse"])[j] > 0 else 0,
            "color" : get_blue(len(rows), Counter(rows["discourse"])[j])
        }
        discourse_sets.append(discourse_set_elements)
    prop_data_packet["discourses"] = discourse_sets
    
    comment_sets = []
    for idx, row in rows.iterrows():
        comment_set_element = {
            "comment_id": row["comment_id"],
            "comment": row["comment"],
            "related_to": row["related_to"],
            "discussion_id": row["discussion_id"],
            "discussion_name": row["discussion_name"],
            "user_id": row["user_id"],
            "emotion": row["emotion"],
            "discourse": row["discourse"],
            "dominant_topic": row["dominant_topic"],
            "topic_phrase": row["topic_phrase"],
            "topic_keywords": row["topic_keywords"],
            "lemmas": row["lemmas"]
        }
        comment_sets.append(comment_set_element)
    prop_data_packet["comments"] = comment_sets
    
    prop_data.append(prop_data_packet)

Sustainable Design and Production
Collection and Management
Sustainable Lifestyle and Education
Research and Innovation
On the Ground Action
Tell Us More


In [5]:
prop_df = pd.DataFrame(prop_data)

In [6]:
# prop_df.to_json('mat_data_cc.json', orient = "records")

prop_df.to_json('mat_data_env.json', orient = "records")

In [7]:
len(prop_df)

6